In [9]:
execfile('../src/utils.py')
execfile('../src/cellslib.py')

In [32]:
configs = [{'path': '/notebooks/moldev-data/original/06-24-2015/ssC HS JS 06.24.2015 LNA.txt',
            'savepath': '/notebooks/moldev-data/analyzed/06-24-2015/ssC LNA.csv',
            'label': 'LNA'},
           {'path': '/notebooks/moldev-data/original/06-24-2015/ssC HS JS 06.24.2015 PNA.txt',
            'savepath': '/notebooks/moldev-data/analyzed/06-24-2015/ssC PNA.csv',
            'label': 'PNA'}]

In [34]:
cell_dict = {"U2OS": ['A01','A02','A03','A04'],
             "143B": ['B01','B02','B03','B04']}

dilutions = [{'Plate Name': [32,34],
              'Concentration': ['2000 nM','1000 nM','500 nM','250 nM']},
             {'Plate Name': [33,35],
              'Concentration': ['125 nM','62.5 nM','31.25 nM','0nM']}]

date = '06-24-2015'

In [36]:
# map(analyze_and_save,configs)

In [11]:
# {path:String, savepath: String, label: String} -> SideEffects(SaveFile)
def analyze_and_save(c):
    """ Clean and save raw data for LNA and PNA data. """
    



In [22]:
cell_types = create_well_df(cell_dict)

In [ ]:
def create_condition_lookup(concentrations):

In [19]:
concentrations_32_34 = ['2000 nM','1000 nM','500 nM','250 nM']
well_numbers = ['01','02','03','04']

conditions = pd.concat([df({'Concentration':concentrations_32_34,
                            '_Well Number': well_numbers,
                            '_Well Letter': well_letter}) \
                           for well_letter in ['A','B']])
conditions['Well Name'] = conditions['_Well Letter'] + conditions['_Well Number']
conditions = conditions.drop(['_Well Letter', '_Well Number'],axis=1)


In [24]:
conditions = pd.merge(conditions,cell_types,on='Well Name')

In [28]:
conditions['Condition'] = 'LNA' + ' ' + conditions['Concentration'] + ' ' + conditions['Cell Type']

In [9]:
well_df = create_well_df(cell_dict)
condition_lookup = pd.concat([create_plate_df(well_df,a,b) for a,b in conditions])
condition_lookup['Date'] = date

In [10]:
condition_lookup.head()

,Well Name,Plate Name,Condition,Plate ID,Date
0,A03,24,PML ssC w RNase A 143B,d4cbc6e79459,06-18-2015
1,A04,24,PML ssC w RNase A 143B,d4cbc6e79459,06-18-2015
2,B03,24,PML ssC w RNase A 143B,d4cbc6e79459,06-18-2015
3,B04,24,PML ssC w RNase A 143B,d4cbc6e79459,06-18-2015
0,A01,24,PML ssC w RNase A U2OS,d4cbc6e79459,06-18-2015


In [11]:
# String -> String
def rename_column(col):
    """ Rename column col to remove whitespace, backslashes, prefixes,
        and suffixes (esp. large parenthetic suffix). """
    if col.startswith('Cell:'):
        return col.split('(')[0].lstrip("Cell:").rstrip('/').strip(' ')
    else:
        return col.split('(')[0].rstrip('/').strip(' ')

cell_config = dict(
    path = path,
    plate_delimiter = "ATF",
    delimiter = '\t',
    skiprows = 4,
    dropcols = ['Cell ID',
                'Site ID',
                'MEASUREMENT SET ID',
                '.*ObjectID.*',
                'Laser focus score',
                '\.[0-9]*\Z'],
    normcols = [['Normalized APB spots (by FITC & TxRed)',
                  ['# of APBs'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized APB spots (by TxRed)',
                  ['# of APBs'],
                  ['# of TxRed spots']],
                ['Normalized APB spots (by FITC)',
                  ['# of APBs'],
                  ['# of FITC spots']],
                
                ['Normalized Coloc avg area (by FITC & TxRed)',
                  ['Area_Coloc_Avg'],
                  ['Area_FITC','Area_TxRed']],
                ['Normalized Coloc avg area (by FITC)',
                  ['Area_Coloc_Avg'],
                  ['Area_FITC']],
                ['Normalized Coloc avg area (by TxRed)',
                  ['Area_Coloc_Avg'],
                  ['Area_TxRed']],                
                
                ['Normalized Coloc spots (by FITC & TxRed)',
                  ['# Coloc Spots'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized Coloc spots (by FITC)',
                  ['# Coloc Spots'],
                  ['# of FITC spots']],
                ['Normalized Coloc spots (by TxRed)',
                  ['# Coloc Spots'],
                  ['# of TxRed spots']],
                
                ['Normalized Coloc total area (by FITC & TxRed)',
                  ['Total_Area_Coloc'],
                  ['Total_Area_FITC','Total_Area_TxRed']],
                ['Normalized Coloc total area (by FITC)',
                  ['Total_Area_Coloc'],
                  ['Total_Area_FITC']],
                ['Normalized Coloc total area (by TxRed)',
                  ['Total_Area_Coloc'],
                  ['Total_Area_TxRed']],
                
                ['Normalized Coloc total area (by FITC-TxRed colocs)',
                  ['Total_Area_Coloc'],
                  ['FITC-TxRed_Area_Total']]],
    colrename = rename_column,
    check = identity
    )

cell_data = get_cell_data(cell_config)

In [12]:
data = pd.merge(cell_data.rename(columns={"Plate ID":"Plate Name"}),
                condition_lookup,
                on = ['Plate Name','Well Name'])

# Write to file
data.to_csv(savepath,index=False)